In [1]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


# MongoDB

In [4]:
from pymongo import MongoClient
import pymongo

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2018 = db.rm_patent_cn_2018
    db_rm_us_2018 = db.rm_patent_us_2018
    db_rm_de_2018 = db.rm_patent_de_2018

    
except errors.ConnectionFailure as err:
    print(err)

# 2018

## Company數量

In [6]:
data_assignee = db_rm_de_2018.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [8]:
data_assignee = db_rm_cn_2018.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_us_2018.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [11]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
2586,Qualcomm Inc,1154
2780,Samsung Electronics Co Ltd,856
3550,Telefonaktiebolaget LM Ericsson AB,510
1684,Intel Corp,471
1699,International Business Machines Corp,350
1539,Huawei Technologies Co Ltd,331
170,Apple Inc,220
700,Cisco Technology Inc,201
2118,Microsoft Technology Licensing LLC,183
3955,Xian Irain IoT Technology Service Co Ltd,177


In [17]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [18]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Qualcomm Inc,1154,7.050989
1,Samsung Electronics Co Ltd,856,6.752270
2,Telefonaktiebolaget LM Ericsson AB,510,6.234411
3,Intel Corp,471,6.154858
4,International Business Machines Corp,350,5.857933
5,Huawei Technologies Co Ltd,331,5.802118
6,Apple Inc,220,5.393628
7,Cisco Technology Inc,201,5.303305
8,Microsoft Technology Licensing LLC,183,5.209486
9,Xian Irain IoT Technology Service Co Ltd,177,5.176150


公司名稱正規化

In [19]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba"))

In [20]:
com_df

,company,count,count_log
0,Qualcomm,1154,7.050989
1,Samsung Electronics,856,6.752270
2,Telefonaktiebolaget LM Ericsson AB,510,6.234411
3,Intel,471,6.154858
4,IBM,350,5.857933
5,Huawei Technologies,331,5.802118
6,Apple,220,5.393628
7,Cisco Technology,201,5.303305
8,Microsoft Technology Licensing LLC,183,5.209486
9,Xian Irain IoT Technology Service,177,5.176150


In [21]:
com_ls = list(com_df.company)

# Data 2018

富比士2000資料

In [22]:
f_data = pd.read_csv("./Forbes/Forbes_2018.csv")

In [23]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Market Value,Revenue,Profits,Assets,Rank,Sector,Industry,Continent,Country,Headquarters,State,CEO,Forbes Webpage,Profits as % of Assets,Profits as % of Revenue,Unnamed: 16
13,Samsung Electronics,325.865,224.644,40.9941,293.154,14.0,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Hyun-Suk Kim,http://www.forbes.com/companies/samsung-electr...,0.139838106,0.182485,NaN
48,Intel,254.762,64.031,11.0910,128.596,49.0,Information Technology,Semiconductors,North America,United States,California,California,Brian Krzanich,http://www.forbes.com/companies/intel/,0.086246851,0.173213,NaN
117,Taiwan Semiconductor,203.024,33.064,11.5138,70.280,118.0,Information Technology,Semiconductors,Asia,Taiwan,Taiwan,NaN,Te Liu,http://www.forbes.com/companies/taiwan-semicon...,0.163827547,0.348228,NaN
199,Broadcom,99.969,18.824,7.7870,54.544,200.0,Information Technology,Semiconductors,North America,United States,California,California,Hock Tan,http://www.forbes.com/companies/broadcom/,0.142765474,0.413674,NaN
200,SK Hynix,56.845,29.288,10.6788,46.088,200.0,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Sung-Wook Park,http://www.forbes.com/companies/sk-hynix/,0.231704565,0.364613,NaN
220,Micron Technology,60.099,25.858,10.0020,41.263,221.0,Information Technology,Semiconductors,North America,United States,Idaho,Idaho,Sanjay Mehrotra,http://www.forbes.com/companies/micron-technol...,0.242396336,0.386805,NaN
411,Texas Instruments,106.947,15.348,4.0170,17.507,412.0,Information Technology,Semiconductors,North America,United States,Texas,Texas,Richard Templeton,http://www.forbes.com/companies/texas-instrume...,0.229451077,0.261728,NaN
438,Applied Materials,57.620,15.463,2.8660,19.663,439.0,Information Technology,Semiconductors,North America,United States,California,California,Gary Dickerson,http://www.forbes.com/companies/applied-materi...,0.145755988,0.185346,NaN
450,ASML Holding,85.378,10.978,2.6925,22.930,451.0,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Peter Wennink,http://www.forbes.com/companies/asml-holding/,0.11742259,0.245263,NaN
559,Qualcomm,81.885,22.605,-4.5560,64.126,560.0,Information Technology,Semiconductors,North America,United States,California,California,Steven Mollenkopf,http://www.forbes.com/companies/qualcomm/,-0.071047625,-0.201548,NaN


In [24]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [29]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [37]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
0,Qualcomm,1154,7.050989
1,Samsung Electronics,856,6.752270
3,Intel,471,6.154858
17,MediaTek,85,4.442651
27,Micron Technology,60,4.094345


In [30]:
# pbc 0.1,0.2
comp_ls = ["Qualcomm","IBM","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB"]

In [31]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [38]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
0,Qualcomm,1154,7.050989
2,Telefonaktiebolaget LM Ericsson AB,510,6.234411
3,Intel,471,6.154858
4,IBM,350,5.857933
5,Huawei Technologies,331,5.802118


In [32]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.07240253043440366
ari:  0.19174193548387097


## 使用Forbes做join

In [33]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [34]:
# pbc 0.1,0.2
comp_ls = ["Qualcomm","IBM","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB"]

In [35]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [36]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.07129962523377328
ari:  0.1134662995367507
